In [4]:
import requests
import json

### UTILS

In [13]:
from typing import List
PYQS_PATH = "data/mar25_pyqs.json"
with open(PYQS_PATH, "r") as f:
    PYQS: List[dict] = json.load(f)
    
ID_TO_PYQ = {pyq["id"]: pyq for pyq in PYQS}
IDX_2_ANS = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)", "(g)", "(h)", "(i)", "(j)"]

def pyqs_to_xml(questions: List[dict]) -> str:
    xml_string = '<questions>\n'

    for i, question_dict in enumerate(questions):
        xml_string += '  <question id="{}">\n'.format(i)
        xml_string += '    <text>{}</text>\n'.format(question_dict["question"])
        xml_string += '    <options>\n'
        
        for i, option in enumerate(question_dict['options']):
            xml_string += '      <option>{}</option>\n'.format(IDX_2_ANS[i] + " " + option)
        
        xml_string += '    </options>\n'
        xml_string += '  </question>\n'

    xml_string += '</questions>'
    return xml_string

def get_current_affairs(query):

    url="https://dev-ai.server.sigiq.ai/async/news-rag/"

    data = {
        "query": query,
    }    

    headers = {'Content-Type': 'application/json'}

    response = requests.get(url, headers=headers, data=json.dumps(data))

    # Checking if the request was successful
    if response.ok:
        # Parsing the JSON response
        response_data = response.json()
        return response_data['response']
    else:
        print("Failed to get data from the endpoint. Status Code:", response.status_code)
        
def pyq_search(query: str, topics: List[str] = [], start_year: int = 1993, end_year: int = 2023) -> str:
    # Endpoint URL
    url = 'https://dev-ai.server.sigiq.ai/async/custom-pyq-retrieval/'

    # JSON body data
    data = {
        "query": query,
        "topics": topics,
        "start_year": start_year,
        "end_year": end_year
    }

    # Headers to indicate JSON content
    headers = {'Content-Type': 'application/json'}

    # Making the GET request with JSON body
    response = requests.get(url, headers=headers, data=json.dumps(data))

    # Checking if the request was successful
    if response.ok:
        # Parsing the JSON response
        response_data = response.json()
                
        # Get all Question IDS from response_data
        question_ids = [elem[0] for elem in response_data['high_relevance_ids']] + [elem[0] for elem in response_data['low_relevance_ids_scores']]
        
        questions = [ID_TO_PYQ[question_id] for question_id in question_ids]
        
        return pyqs_to_xml(questions)
    else:
        print("Failed to get data from the endpoint. Status Code:", response.status_code)
        return ""

    

In [17]:
result = get_current_affairs("South China Sea")
print(result)

### South China Sea Disputes and Incidents

#### Overview
The South China Sea has been a region of heightened tensions due to overlapping territorial claims by China, the Philippines, Vietnam, Malaysia, Taiwan, and Brunei. The strategic and economically significant waterway has seen a series of confrontations, particularly between China and the Philippines.

#### Key Incidents and Responses

- **October 23, 2023**: Chinese and Philippine vessels collided near Second Thomas Shoal, with accusations of dangerous maneuvers and water cannon use by the Chinese Coast Guard.
- **November 2023**: A Chinese coast guard ship and accompanying vessels conducted dangerous maneuvers, blocking and colliding with a Philippine supply boat, causing injuries and damage.
- **March 2024**: Chinese coast guard ships attempted to block Philippine government vessels carrying scientists for marine and biodiversity research at Sandy Cay. Despite warnings, the Filipino team completed their research.
- **March 202

In [33]:
def book_rag(query: str):
    url="https://dev-ai.server.sigiq.ai/async/book-rag/book_rag_for_agent/"

    data = {
        "book": "polity_laxmikanth",
        "query": query,
    }    

    headers = {'Content-Type': 'application/json'}

    response = requests.get(url, headers=headers, data=json.dumps(data), timeout=180)

    # Checking if the request was successful
    if response.ok:
        # Parsing the JSON response
        return response
    else:
        print("Failed to get data from the endpoint. Status Code:", response.status_code)
        return response


In [37]:
book_rag_resp = book_rag(query="Are constitutional amendments under Art 368 considered laws within the meaning of Art 13?")

In [38]:
resp = book_rag_resp.json()['response']

In [42]:
resp[1]

{'text': "Furthermore, the book elaborates that *Article 368* provides the procedure for the amendment of the Constitution and explicitly states that the Parliament may, in exercise of its constituent power, amend by way of addition, variation, or repeal any provision of the Constitution. However, the Supreme Court in the *Kesavananda Bharati case* also ruled that the constituent power of Parliament under Article 368 does not enable it to alter the 'basic structure' of the Constitution. This implies that while Parliament has the power to amend the Constitution, this power is not absolute and is subject to certain inherent limitations, which are not defined under Article 13 but are part of the 'basic structure' doctrine.",
 'locations': [{'chapter': 'amendment of the constitution',
   'chapter_num': 10,
   'section': 'procedure for amendment',
   'subsection': ''},
  {'chapter': 'basic structure of the constitution',
   'chapter_num': 11,
   'section': 'elements of the basic structure',

In [49]:
def format_answer_xml(answer_components):
    xml = '<answer>\n'
    
    for component in answer_components:
        xml += '  <component>\n'
        xml += f'    <text>{component["text"]}</text>\n'
        xml += '    <locations>\n'
        for location in component['locations']:
            xml += '      <location>\n'
            xml += f'        <chapter>{location["chapter"]}</chapter>\n'
            xml += f'        <chapter_num>{location["chapter_num"]}</chapter_num>\n'
            xml += f'        <section>{location["section"]}</section>\n'
            xml += f'        <subsection>{location["subsection"]}</subsection>\n'
            xml += '      </location>\n'
        xml += '    </locations>\n'
        xml += '  </component>\n'
    
    xml += '</answer>'
    return xml

In [50]:
print(format_answer_xml(resp))

<answer>
  <component>
    <text>The question of whether constitutional amendments under Article 368 are considered 'laws' within the meaning of Article 13 has been a subject of judicial scrutiny. According to the book, **Article 13** declares that all laws that are inconsistent with or in derogation of the Fundamental Rights shall be void. In this context, 'law' includes not only legislation but also ordinances, orders, bye-laws, rules, regulations, and notifications. However, the Supreme Court held in the *Kesavananda Bharati case (1973)* that a Constitutional amendment is not a 'law' within the meaning of Article 13. Therefore, the amendments to the Constitution made under Article 368 are not subject to the same scrutiny as ordinary laws under Article 13.</text>
    <locations>
      <location>
        <chapter>judicial review</chapter>
        <chapter_num>27</chapter_num>
        <section>meaning of judicial review</section>
        <subsection></subsection>
      </location>
    